In [1]:
import sqlite3
conn = sqlite3.connect("Db-IMDB.db")
#cur = conn.cursor()
import pandas as pd

### 1. List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [2]:
sql_query='''SELECT p.name,m.title, m.year FROM MOVIE m, M_Genre g,Genre n,M_Director d, Person p
                           where trim(year) % 4 = 0 \
                           and  m.MID=g.MID \
                           and  m.MID=d.MID \
                           and  d.PID=p.PID \
                            and g.GID=n.GID \
                            and trim(n.Name)='Comedy' '''

result = pd.read_sql_query( sql_query, conn)
result

,Name,title,year
0,Milap Zaveri,Mastizaade,2016
1,Milap Zaveri,Mastizaade,2016
2,Umesh Ghadge,Kyaa Kool Hain Hum 3,2016
3,Umesh Ghadge,Kyaa Kool Hain Hum 3,2016
4,Sameer Sharma,Luv Shuv Tey Chicken Khurana,2012
5,Anees Bazmee,Thank You,I 2011
6,Anees Bazmee,Thank You,I 2011
7,Sanjeev Sharma,Saat Uchakkey,2016
8,Sanjeev Sharma,Saat Uchakkey,2016
9,Nitin Kakkar,Filmistaan,2012


### 2. List the names of all the actors who played in the movie 'Anand' (1971)

In [3]:
sql_query='''SELECT p.Name FROM Movie m, M_Cast c, Person p \
                           where  trim(title) = 'Anand' \
                           and m.MID=c.MID \
                           and trim(c.PID)=trim(p.PID)  '''

result = pd.read_sql_query( sql_query, conn)
result

,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo
5,Asit Kumar Sen
6,Dev Kishan
7,Atam Prakash
8,Lalita Kumari
9,Savita


### 3. List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [6]:
#https://stackoverflow.com/questions/26522727/convert-string-to-int-inside-where-clause-of-sqlite-statment
sql_query='''SELECT Distinct P.Name FROM Movie m, M_Cast c, Person p 
                           where cast(substr(year,-4) as INTEGER) < 1970 
                           and m.MID = c.MID 
                           and trim(c.PID) = p.PID
                           and trim(p.pid) in (SELECT distinct trim(p.pid) FROM Movie m, M_Cast c, Person p 
                           where cast(substr(year,-4) as INTEGER) > 1990 
                           and m.MID = c.MID 
                           and trim(c.PID) = p.PID)
                           order by P.name
                            '''

result = pd.read_sql_query( sql_query, conn)
result 

,Name
0,A.K. Hangal
1,Aachi Manorama
2,Abbas
3,Abdul
4,Abhi Bhattacharya
5,Achala Sachdev
6,Ajay
7,Ajit
8,Akbar Bakshi
9,Alka


### 4. List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [21]:
sql_query='''SELECT p.name,count(p.name) AS 'No of Movies' FROM Movie m, M_Director d, Person p \
                        where m.MID = d.MID \
                        and trim(d.PID) = p.PID 
                        group by p.name 
                        having count(p.name) >=10
                        ORDER BY count(p.name) DESC ;   '''

result = pd.read_sql_query( sql_query, conn)
result 

,Name,No of Movies
0,David Dhawan,39
1,David Dhawan,39
2,Mahesh Bhatt,36
3,Mahesh Bhatt,36
4,Ram Gopal Varma,30
5,Priyadarshan,30
6,Ram Gopal Varma,30
7,Vikram Bhatt,29
8,Vikram Bhatt,29
9,Hrishikesh Mukherjee,27


### 5 a. For each year, count the number of movies in that year that had only female actors.

In [8]:
sql_query='''SELECT  distinct substr(year,-4) as 'Year', count(substr(year,-4)) AS 'No of Movies' FROM Movie m, M_Cast c, Person p \
                        where m.MID = c.MID 
                        and trim(c.PID) = p.PID 
                        and m.mid not in ( SELECT distinct m.mid FROM Movie m, M_Cast c, Person p 
                        where m.MID = c.MID 
                        and trim(c.PID) = p.PID 
                        and p.gender = 'Male'   )
                        Group by substr(year,-4) ;   '''

result = pd.read_sql_query( sql_query, conn)
result 

,Year,No of Movies
0,1939,2
1,1999,11
2,2000,11
3,2018,3


### 5 b. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [14]:
sql_query='''Select distinct m.year as 'Year',m.TMovies as 'Total Movies', ((a.FMovies * 100.00) / m.TMovies) as '%age of Female Movies'
                from  (SELECT substr(m.year,-4) as 'Year', count(m.year) AS 'TMovies' FROM Movie m, M_Cast c, Person p \
                        where m.MID = c.MID \
                        and trim(c.PID) = p.PID \
                        Group by substr(m.year,-4)) m\
                Join ( SELECT  distinct substr(m.year,-4) as 'Year', count(substr(year,-4)) AS 'FMovies' FROM Movie m, M_Cast c, Person p \
                        where m.MID = c.MID 
                        and trim(c.PID) = p.PID 
                        and m.mid not in ( SELECT distinct m.mid FROM Movie m, M_Cast c, Person p 
                        where m.MID = c.MID 
                        and trim(c.PID) = p.PID 
                        and p.gender = 'Male'   )
                        Group by substr(m.year,-4)) a on m.year=a.year
                        
                group by m.year;   '''

result = pd.read_sql_query( sql_query, conn)
result 

,Year,Total Movies,%age of Female Movies
0,1939,46,4.347826
1,1999,1709,0.643651
2,2000,1578,0.697085
3,2018,3009,0.099701


### 6. Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [25]:
sql_query=''' SELECT m.title,b.Total_Cast FROM Movie m
                     Left Join (SELECT c.MID,Count(a.PID) as 'Total_Cast' FROM M_Cast c
                                        Left Join (Select Distinct p.PID from Person p) a
                                                    where trim(c.PID) = a.PID 
                                                        group by c.MID) b
               WHERE m.MID=b.MID
               order by b.Total_Cast DESC
            ;   '''

result = pd.read_sql_query( sql_query, conn)
result 

,title,Total_Cast
0,Ocean's Eight,238
1,Apaharan,233
2,Gold,215
3,My Name Is Khan,213
4,Captain America: Civil War,191
5,Geostorm,170
6,Striker,165
7,2012,154
8,Pixels,144
9,Yamla Pagla Deewana 2,140


### 7. A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [17]:
#https://stackoverflow.com/questions/51609285/sql-query-for-find-the-decade-with-the-largest-number-of-records
#sql_query='''select substr(year,-4),count(year) from movie  group by year '''
sql_query='''select y.year as decade_start, y.year + 9 as decade_end, count(*) as num_movies
                from (select distinct year from movie) y 
                join movie m on m.year >= y.year and m.year < y.year + 10
                group by y.year
                order by count(*) desc
                limit 1;
              '''
result = pd.read_sql_query( sql_query, conn)
result 

,decade_start,decade_end,num_movies
0,2008,2017,1128


### 8. Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies).

In [12]:
sql_query='''select p.name from Movie m,M_Cast c,Person p 
                    where m.mid=c.mid
                    and trim(c.pid)=p.pid        
                    and p.name not in ( select  distinct a.name from (select m.year,p.name from Movie m,M_Cast c,Person p 
                                        where m.mid=c.mid
                                        and trim(c.pid)=p.pid        
                                        order by p.Name,m.year) a 
                                join (select  m.year,p.name from Movie m,M_Cast c,Person p 
                                        where m.mid=c.mid
                                        and trim(c.pid)=p.pid        
                                        order by p.Name,m.year) b on a.name=b.name
                                        where a.year-b.year > 3 )
                    
                    '''
result = pd.read_sql_query( sql_query, conn)
result 

,Name
0,Christian Bale
1,Cate Blanchett
2,John Benfield
3,Lorna Brown
4,Patrick Godfrey
5,Naomie Harris
6,Tom Hollander
7,Eddie Marsan
8,Peter Mullan
9,Matthew Rhys


### 9. Find all the actors that made more movies with Yash Chopra than any other director.

In [27]:
sql_query='''select distinct a.name from ( 
                                select distinct p.pid,p.name,d.pid, count(*) as 'Total' from Movie m,M_Cast c,Person p , M_Director d
                                    where m.mid=c.mid
                                    and m.mid=d.mid
                                    and trim(c.pid)=p.pid
                                    and d.pid not in ('nm0007181')
                                    and p.pid in ( select distinct p.pid from Movie m,M_Cast c,Person p 
                                                where m.mid=c.mid
                                                and trim(c.pid)=p.pid  
                                                and m.mid in ( select m.mid from Movie m,M_Director d,Person p 
                                                                where m.mid=d.mid
                                                                and trim(d.pid)=p.pid  
                                                                and p.name like 'Yash%' )) group by p.pid,p.name,d.pid ) a 
                    
                            join (select distinct p.pid,count(trim(p.pid)) as 'Yash' from Movie m,M_Cast c,Person p 
                                    where m.mid=c.mid
                                    and trim(c.pid)=p.pid  
                                    and m.mid in ( select m.mid from Movie m,M_Director d,Person p 
                                                    where m.mid=d.mid
                                                    and trim(d.pid)=p.pid  
                                                    and p.name like 'Yash%' )
                                    group by p.pid) b on a.pid=b.pid
                            where  b.Yash > a.Total 
                            
                                   
                    '''
result = pd.read_sql_query( sql_query, conn)
result 

,name
0,Amitabh Bachchan
1,Gulshan Grover
2,Shashi Kapoor
3,Shashi Kapoor
4,Rajesh Khanna
5,Sridevi
6,Juhi Chawla
7,Hema Malini
8,Hema Malini
9,Sunil Dutt


### 10. The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [16]:
sql_query =''' select distinct p.name from Movie m,M_Cast c,Person p 
                                    where m.mid=c.mid
                                    and trim(c.pid)=p.pid
                                    and m.mid in (select distinct m.mid from Movie m,M_Cast c,Person p 
                                                where m.mid=c.mid
                                                and trim(c.pid)=p.pid
                                                and p.pid in ( select distinct p.pid from Movie m,M_Cast c,Person p 
                                                               where m.mid=c.mid
                                                               and trim(c.pid)=p.pid
                                                               and m.mid in ( select distinct m.mid from Movie m,M_Cast c,Person p 
                                                                              where m.mid=c.mid
                                                                              and trim(c.pid)=p.pid  
                                                                              and p.name like '%Shah Rukh Khan%')))
                '''
result = pd.read_sql_query( sql_query, conn)
result 

,Name
0,Kristin Scott Thomas
1,Derek Jacobi
2,Nick Frost
3,Walton Goggins
4,Michael Obiora
5,Dominic West
6,Alexandre Willaume
7,Daniel Wu
8,Josef Altin
9,Jaime Winstone


In [ ]:
con.close()